In [3]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [ ]:
df = pd.read_csv('Sources/fiuba_6_avisos_detalle_concat.csv', delimiter=',')
tuples = [tuple(x) for x in df.values]
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

In [ ]:
for x in tuples:
    x = tuple(x)

In [ ]:
avisos = sc.parallelize(tuples)
avisos.take(1)

In [ ]:
avisos = avisos.map(lambda x: (x[0],x[1],x[2],str.upper(re.sub("<.*?>", " ", x[3])),str.upper(re.sub("<.*?>", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11])).map(lambda x: (x[0],x[1],x[2],str(re.sub("\xa0", " ", x[3])),str(re.sub("\xa0", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))
avisos.take(1)

In [ ]:
import re, string, unicodedata
def removePunctuation(text):
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

In [ ]:
def palabrasLargas(texto):
    lista = texto.split(' ') 
    lista = [x for x in lista if not len(x)>17]
    return " ".join(lista)

In [ ]:
avisos.take(1)

In [ ]:
avisos = avisos.map((lambda x: (x[0],x[1],x[2],removePunctuation(x[3]),removePunctuation(x[4]),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))).map((lambda x: (x[0],x[1],x[2],palabrasLargas(x[3]),x[4],x[5],x[6],x[7],x[8], x[9],x[10],x[11]))).collect()

In [ ]:
avisos = pd.DataFrame.from_records(avisos).drop([0], axis=1)

In [ ]:
for x in avisos.columns:  
    avisos = avisos.rename(columns = {x:df.columns[x-1]})
avisos.head()
avisos = avisos.drop(['idpais','nombre_zona','mapacalle', 'ciudad'], axis=1)
avisos.head()

In [4]:
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv')

In [5]:
cluster = pd.read_csv('idaviso-cluster.csv')

In [6]:
avisos.drop('Unnamed: 0',axis=1,inplace=True)
cluster.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
cluster.head()

,idaviso,cluster
0,1111556097,10
1,1111949392,10
2,1112145935,10
3,1112146010,10
4,1112211475,10


In [8]:
avisos.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,responsable de marketingproducto div construcc...,buscamos un responsable de producto para inc...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet u...,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,software engineer,chevron is a multinational company in the ener...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,business developer manager,importante empresa de energias renovables se e...,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [9]:
avisos = avisos.merge(cluster,on='idaviso')

In [10]:
avisos.sample()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster
4357,1112226812,senior de impuestos,pgk consultores firma miembro de tgs global ne...,Full-time,Senior / Semi-Senior,Impuestos,PGK Consultores,5


In [11]:
avisos.to_csv("Sources/fiuba_6_avisos_detalle_limpio.csv")